# Условная оптимизация

В предыдущей задаче мы рассматривали задачу с автомобиллем и полем. В ней не было никаких ограничений (собственно, в ней не было *явных* ограничений, например, *подразумевалось*, что автомобиль не выезжал за пределы поля). 

Такие задачи называются задачами безусловной оптимизации.

Однако, в реальности могут ситуации, когда решение придется ограничить. Например, в той же задаче с автомобилем и полем у нас в автомобиле может быть ограниченное количество топлива, и нам по-прежнему необходимо пересечь поле за минимальное время, но так, чтобы нам хватило топлива.

Собственно, в реальности такие задачи тоже возникают регулярно, например, в космической отрасли мы можем столкнуться с задачей выведения космического корабля на орбиту за минимальное время и с ограниченным запасом топлива.

## Математическая постановка задачи

Пусть дана функция $f(x_0, x_1, \dots, x_n)$, зависящая от переменных $x_0, x_1, \dots x_n$.

Требуется минимизировать функцию $f(x_0, x_1, \dots, x_n)$:

$$
\min_{x_0, x_1, \dots, x_n} f(x_0, x_1, \dots, x_n)
$$

при выполнении условий 
$$
g_i(x_0, x_1, \dots, x_n) \ge 0
$$
$$
h_j(x_0, x_1, \dots, x_n) = 0
$$

При работе в пакете SciPy подход к записи ограничений несколько иной: все ограничения записываются в виде

$$
c^L \le g_i(x) \le c^U
$$

и 

$$
x^L \le x \le x^U
$$

где $c^L$ и $x^L$ - нижние границы (константа в том и в другом случае), $c^U$ и $x^U$ - верхние границы.

Сама функция для поиска минимума функции будет та же, `minimize`, а ограничения задаются через специальные объекты: `Bounds` для ограничений на переменные $x$, `LinearConstraint` для линейных функций-ограничений ($g(x)$), и `LinearConstraint` для нейлинейных (более подробно см в [статье по условной оптимизации](https://habr.com/ru/company/ods/blog/448054/)).

Рассмотрим решение задачи на примере:

необходимо найти минимум функции 
$$
f(x) = 3 \cdot (x - 1) ^ 2 - 2
$$

при условии, что 
$$
x - 1 \ge 2
$$

Учитывая, что условие
$$
x - 1 \ge 2
$$

можно переписать как

$$
2 \le x - 1 \le \infty
$$

или, еще более формально,

$$
2 \le 1 \cdot x - 1 \le \infty
$$


Для того, чтобы правильно записать это ограничение, необходимо вспомнить матричную форму записи линейных алгебраических уравнений (см., например, [здесь](https://www.youtube.com/watch?v=yoG80Lf5Wik))

К сожалению, функция `minimize` по умолчанию не умеет работать с ограничениями, и при запуске выдает ошибку. Для работы с ограничениями необходимо указать функции `minimize` использовать специальные методы. 

Функция `minimize` поддерживает три метода ооптимизации, для которых можно задать ограничения:

* [COBYLA](https://en.wikipedia.org/wiki/COBYLA)
* [SLSQP](https://ru.wikipedia.org/wiki/Последовательное_квадратичное_программирование)
* [trust-constr](https://en.wikipedia.org/wiki/Trust_region)

Рассмотрим пример:

In [6]:
import numpy as np  # нам понадобится пакет numpy, чтобы использовать значение бесконечность (np.inf)
import math  # 
from scipy.optimize import minimize  # берем готовую библиотечную функцию minimize
from scipy.optimize import LinearConstraint  # будем использовать линейные ограничения

# описываем функцию, которую хотим минимизировать
def func_to_minimize(x):
    return 3 * math.pow(x - 1, 2) - 2
    
# задаем линейное ограничение
constraint = LinearConstraint ([[1]], [3], [np.inf])  # первая матрица - это коэффициенты при x, в нашем случае - единица
# вторая матрица - нижние границы, в нашем случае - 2, третья матрица - верхние ограничения
# в нашем случае верхних ограничений нет, а занчит, верхнее ограничение равно бесконечности

# Зададим начальное значение x
x = 4  # подумайте, почему начально значение в данном случае равно трем

# а теперь минимизируем нашу функцию
result = minimize(func_to_minimize, x, method='trust-constr', constraints=[constraint])
print(result)  # выведем наш результа на экран

           message: `gtol` termination condition is satisfied.
           success: True
            status: 1
               fun: 10.00401061178826
                 x: [ 3.000e+00]
               nit: 8
              nfev: 12
              njev: 6
              nhev: 0
          cg_niter: 5
      cg_stop_cond: 1
              grad: [ 1.200e+01]
   lagrangian_grad: [ 3.658e-09]
            constr: [array([ 3.000e+00])]
               jac: [array([[ 1.000e+00]])]
       constr_nfev: [0]
       constr_njev: [0]
       constr_nhev: [0]
                 v: [array([-1.200e+01])]
            method: tr_interior_point
        optimality: 3.658341185541758e-09
  constr_violation: 0.0
    execution_time: 0.01889514923095703
         tr_radius: 175.0
    constr_penalty: 1.0
 barrier_parameter: 0.004000000000000001
 barrier_tolerance: 0.004000000000000001
             niter: 8


## Задание для самостоятельной работы

Изменить код в ячейке ниже (код основан на предыдущей работе) таким образом, чтобы учитывлось ограничение на время движения (необходимо затратить не более 0.028 часа), и расход топлива был минимальным

In [5]:
import math  # 
from scipy.optimize import minimize  # берем готовую библиотечную функцию minimize

# задаим исходные данные в километрах

h1 = 0.10  # 100 метров
h2 = 0.10  # 100 метров
l = 1  # 1000 метров

v1 = 40  # 40 км/ч
v2 = 30  # 30 км/ч

c1 = 0.115  # л/км
c2 = 0.15  # л/км

# описываем функцию, которую хотим минимизировать

def func_to_minimize(x):
    return # вставьте вашу функцию здесь
    
# Зададим начальное значение x, пусть это будет самое начало поля, т.е., вначале мы поедем вериткально вверх,
# а затем, как только достигнем конца засеянной части, кратчайшим путем поедем в правый верхний угол поля
x = 0

# а теперь минимизируем нашу функцию
result = minimize(func_to_minimize, x)
print(result)  # выведем наш результа на экран

TypeError: '<' not supported between instances of 'NoneType' and 'float'